In [1]:
# Windiws Forms Photoplethysmogram for PPG data acquision# Created by Dusan Radivojevic, 11.2020# Libraries
import tkinter as tk
from tkinter import *
import os
import serial.tools.list_ports as lisCom
import serial
global portName
import time
import matplotlib
matplotlib.use('TkAgg')
import numpy as np
from matplotlib.backends.backend_tkagg import FigureCanvasTkAgg
from matplotlib.figure import Figure
import sys
sys.path.insert(0,'..')
from ppg import spo2class

In [64]:
# Root of application
root=tk.Tk()
root.title("PPG Acquisition System")
root.resizable(0,0)
# USB connection method
hwIde='VID:PID=0000:0400'
portName='COM3'
duration=60
Fl=0
indexIR=[]
indexRed=[]
middleIR=[]
middleRed=[]
ringIR=[]
ringRed=[]
global HR, SPO2, PI
# Connect to USB
def Usb():
    ports=lisCom.comports()
    portNames=[]
    portString=[]
    for p,d,h in sorted (ports):
        portString.append("{}: {} [{}]".format(p,d,h))
        portNames.append(p)
        for i in range(len(h)-17):
            if(h[i:i+17]==hwIde):
                portName=p
    # Open serial port
    global serialPort
    try:
        serialPort = serial.Serial(port = portName, baudrate=115200,
                               bytesize=8, timeout=2, stopbits=serial.STOPBITS_ONE)
        usbConnect['bg']='green'
    except :
        usbConnect['bg']='red'
# PPG measurement method
def PPGMeasurement():
    # Software reset of serial port
    serialPort.close()
    serialPort.open()
    # Measurement
    T0=time.time()
    my_list=[]
    while(T0>(time.time()-(duration+4))):
        # Wait until there is data waiting in the serial buffer
        if(serialPort.in_waiting > 0):
            # Read data out of the buffer and put them in the list
            serialString = serialPort.readline()
            serialString1=str(serialString.decode('Ascii'))
            my_list.append(serialString1.split(","))  
    # Convert to intiger array
    my_list_Int=[]
    cupple=[]
    for i in range (len(my_list)):
        try:
            if (len(my_list[i])==2):
                cupple.append(int(my_list[i][0]))
                cupple.append(int(my_list[i][1]))
                my_list_Int.append(cupple)
                cupple=[]
        except ValueError:
            cupple=[]
            i+=1  
    # Reject small values
    my_new_Int_List=[]
    for i in range(len(my_list_Int)):
        if ((my_list_Int[i][0]>10000) and (my_list_Int[i][1]>10000)):
            my_new_Int_List.append(my_list_Int[i])
    # Separate and cut down arrays
    IR_array=[]
    Red_array=[]
    for i in range (50*duration):
        IR_array.append(my_new_Int_List[i][0])
        Red_array.append(my_new_Int_List[i][1])
    # HR, SPO2, PI
    b=spo2class(IR_array,Red_array)
    HR, SPO2, PI = b.run()
    if SPO2>100:
        SPO2=100
    label11=tk.Label(root,text='-HR\t\t'+str(HR),bg="#9da185",fg="white",font=("Helvetica", 12))
    label11.place(x=20,y=335)
    label11=tk.Label(root,text='-SPO2\t\t'+str(SPO2),bg="#9da185",fg="white",font=("Helvetica", 12))
    label11.place(x=20,y=357)
    label11=tk.Label(root,text='-PI\t\t'+str(PI),bg="#9da185",fg="white",font=("Helvetica", 12))
    label11.place(x=20,y=379)
    # Plot graph
    global Fl,canvas1
    fig = Figure(figsize=(16,8))
    a = fig.add_subplot(111)
    a.plot(range(len(IR_array)),IR_array,color='blue')
    a.plot(range(len(Red_array)),Red_array,color='red')
    a.set_title ("Sensor signals", fontsize=12)
    a.set_ylabel("A/D value", fontsize=11)
    if Fl>0:
        canvas1.get_tk_widget().pack_forget()
    canvas1 = FigureCanvasTkAgg(fig, frame)
    canvas1.get_tk_widget().place(x=35, y=0,height=340,width=600)
    canvas1.draw()
    Fl+=1
    # Define arrays
    global indexIR,indexRed,middleIR,middleRed,ringIR,ringRed,IHR,MHR,RHR,ISPO2,MSPO2,RSPO2,IPI,MPI,RPI
    if Fl==1:
        indexIR=IR_array
        indexRed=Red_array
        indexFinger['bg']='green'
        IHR=HR
        ISPO2=SPO2
        IPI=PI
    else:
        if Fl==2:
            middleIR=IR_array
            middleRed=Red_array
            middleFinger['bg']='green'
            MHR=HR
            MSPO2=SPO2
            MPI=PI
        else:
            ringIR=IR_array
            ringRed=Red_array
            ringFinger['bg']='green'
            RHR=HR
            RSPO2=SPO2
            RPI=PI
# Erase last measurement
def Erase():
    global Fl
    if Fl==1:
        indexFinger['bg']='#263D42'
    else:
        if Fl==2:
            middleFinger['bg']='#263D42'
        else:
            ringFinger['bg']='#263D42'
    if Fl>0:
        Fl-=1
# Closing serial port
def on_closing():
    try:
        serialPort.close()
    except:
        print('Serial close exeption!')
    root.destroy()
# Save all data to txt file with CSV format
def Save():
    try:
        with open(entry1.get()+".txt", "w") as txt_file:
            txt_file.write("ID, IR, Red, Fing, Gend, Age, Heig, Weig, Sys, Dia, HR_BP, HR_PPG, SPO2, PI")
            for i in range(3000):
                txt_file.write('\n'+entry1.get()+', '+str(indexIR[i])+', '+str(indexRed[i])+', I, '+str(listbox.curselection()[0])
                               +', '+entry2.get()+', '+entry3.get()+', '+entry4.get()+', '+entry5.get()+', '+entry6.get()+', '
                               +entry7.get()+', '+str(IHR)+', '+str(ISPO2)+', '+str(IPI))
            for i in range(3000):
                txt_file.write('\n'+entry1.get()+', '+str(middleIR[i])+', '+str(middleRed[i])+', M, '+str(listbox.curselection()[0])
                               +', '+entry2.get()+', '+entry3.get()+', '+entry4.get()+', '+entry5.get()+', '+entry6.get()+', '
                               +entry7.get()+', '+str(MHR)+', '+str(MSPO2)+', '+str(MPI))
            for i in range(3000):
                txt_file.write('\n'+entry1.get()+', '+str(ringIR[i])+', '+str(ringRed[i])+', R, '+str(listbox.curselection()[0])
                               +', '+entry2.get()+', '+entry3.get()+', '+entry4.get()+', '+entry5.get()+', '+entry6.get()+', '
                               +entry7.get()+', '+str(RHR)+', '+str(RSPO2)+', '+str(RPI))
        saveAll['bg']='green'
    except:
        tk.messagebox.showerror(title='Save error!',message='An error occurred!')
# Window def
canvas=tk.Canvas(root,height=405,width=900)
canvas.pack()
frame0=tk.Frame(root,bg="#767676")
frame0.place(relwidth=0.996,relheight=0.99,relx=0.002,rely=0.005)
frame=tk.Frame(root,bg="white")
frame.place(relwidth=0.7,relheight=0.85,relx=0.298,rely=0.005)
frame1=tk.Frame(root,bg="#9da185")
frame1.place(relwidth=0.996,relheight=0.18,relx=0.002,rely=0.815)
# Button for USB connection
usbConnect=tk.Button(root,text="Connect USB",padx=1,pady=1,fg='white',bg='#263D42')
usbConnect['command']=Usb
usbConnect.place(x=35,y=15,height=30,width=200)
# Info label
label1=tk.Label(root,text='Enter subject data for:',bg="#767676",fg="white",font=("Helvetica", 12))
label1.place(x=55,y=50)
# Entry for ID number
label2=tk.Label(root,text='- ID',bg="#767676",fg="white",font=("Helvetica", 12))
label2.place(x=20,y=75)
entry1 =tk.Entry(root,bg='#29202E',fg='white',justify="center")
entry1.place(x=120,y=78,height=20,width=120)
# Gender listbox
label3=tk.Label(root,text='- Gender',bg="#767676",fg="white",font=("Helvetica", 12))
label3.place(x=20,y=105)
listbox=tk.Listbox(root,bg='#29202E',fg='white',justify="center",selectbackground='#263D42')
listbox.insert(END,"Man")
listbox.insert(END,"Woman")
listbox.place(x=120,y=108,height=20,width=120)
# Age Entery
label4=tk.Label(root,text='- Age',bg="#767676",fg="white",font=("Helvetica", 12))
label4.place(x=20,y=135)
entry2=tk.Entry(root,bg='#29202E',fg='white',justify="center")
entry2.place(x=120,y=138,height=20,width=120)
# Height Entery
label5=tk.Label(root,text='- Height',bg="#767676",fg="white",font=("Helvetica", 12))
label5.place(x=20,y=165)
entry3=tk.Entry(root,bg='#29202E',fg='white',justify="center")
entry3.place(x=120,y=168,height=20,width=120)
# Weight Entery
label6=tk.Label(root,text='- Weight',bg="#767676",fg="white",font=("Helvetica", 12))
label6.place(x=20,y=195)
entry4=tk.Entry(root,bg='#29202E',fg='white',justify="center")
entry4.place(x=120,y=198,height=20,width=120)
# Systolic Blood Pressure Entery
label7=tk.Label(root,text='- Systolic',bg="#767676",fg="white",font=("Helvetica", 12))
label7.place(x=20,y=225)
entry5=tk.Entry(root,bg='#29202E',fg='white',justify="center")
entry5.place(x=120,y=228,height=20,width=120)
# Diastolic Blood Pressure Entery
label8=tk.Label(root,text='- Diastolic',bg="#767676",fg="white",font=("Helvetica", 12))
label8.place(x=20,y=255)
entry6=tk.Entry(root,bg='#29202E',fg='white',justify="center")
entry6.place(x=120,y=258,height=20,width=120)
# Heart Rate Entery
label9=tk.Label(root,text='- Heart Rate',bg="#767676",fg="white",font=("Helvetica", 12))
label9.place(x=20,y=285)
entry7=tk.Entry(root,bg='#29202E',fg='white',justify="center")
entry7.place(x=120,y=288,height=20,width=120)
# PPG Index Finger record
label10=tk.Label(root,text='Measure:',bg="#9da185",fg="white",font=("Helvetica", 12))
label10.place(x=290,y=340)
label14=tk.Label(root,text='Data:',bg="#9da185",fg="white",font=("Helvetica", 12))
label14.place(x=290,y=373)
indexFinger=tk.Button(root,text="Index Finger",padx=1,pady=1,fg='white',bg='#263D42')
indexFinger['command']=PPGMeasurement
indexFinger.place(x=380,y=340,height=30,width=110)
# PPG Middle Finger record
middleFinger=tk.Button(root,text="Middle Finger",padx=1,pady=1,fg='white',bg='#263D42')
middleFinger['command']=PPGMeasurement
middleFinger.place(x=580,y=340,height=30,width=110)
# PPG Ring Finger record
ringFinger=tk.Button(root,text="Ring Finger",padx=1,pady=1,fg='white',bg='#263D42')
ringFinger['command']=PPGMeasurement
ringFinger.place(x=780,y=340,height=30,width=110)
# Erase record
eraseRecord=tk.Button(root,text="Erase record",padx=1,pady=1,fg='white',bg='#263D42')
eraseRecord['command']=Erase
eraseRecord.place(x=380,y=373,height=30,width=110)
# On closing go to method for closing serial port too
c=root.protocol("WM_DELETE_WINDOW", on_closing)
# Save all records and data
saveAll=tk.Button(root,text="Save All",padx=1,pady=1,fg='white',bg='#263D42')
saveAll['command']=Save
saveAll.place(x=580,y=373,height=30,width=110)
root.mainloop()